In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras import layers
from keras.layers import Dropout
from scipy import stats
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.
/home/sasthana/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sasthana/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sasthana/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sasthana/.local/lib/python3.6/site-packages/tensorflo

In [1]:
path = "../../../mnt/storage/Datasets/final_data/"
hip = pd.read_csv(path+"Hip.csv", low_memory = False)
wrist = pd.read_csv(path+"Wrist.csv", low_memory = False)
thigh = pd.read_csv(path+"Thigh.csv", low_memory = False)
chest = pd.read_csv(path+"Chest.csv", low_memory=False)

NameError: name 'pd' is not defined

To process the data in the correct shape for RNN, we will group the data into sequential time intervals. So the input for the model will be a 3-d array with each subarray representing a sequence of n seconds.

In [10]:
def clean_data(data, halved = False):
    data.drop([col for col in data.columns if "Unnamed" in col], axis=1, inplace = True)
    # Drop un-encoded data
    data = data[data['updated_final_activity'] != 'private/not coded']
    if not halved:
        return data
    else:
        data_halved = data[data.updated_final_activity != 'sit/lie']
        data2 = data[data.updated_final_activity == 'sit/lie'].sample(frac = 0.5)
        data_halved = data_halved.append(data2)
    return data_halved
hip = clean_data(hip)
wrist = clean_data(wrist)
thigh = clean_data(thigh)
chest = clean_data(chest)

In [11]:
x_cols = pd.read_csv('columns.csv')
x_cols = x_cols['0'].to_list()
activities = wrist.updated_final_activity.unique()
activities
hip.shape, wrist.shape, thigh.shape, chest.shape

((257300, 107), (257300, 107), (203912, 107), (271896, 107))

In [5]:
def aggregate(data,step=135):
    agged = []
    agged_y = []
    x_cols = pd.read_csv('columns.csv')
    x_cols = x_cols['0'].to_list()
    activities = data.updated_final_activity.unique()

    X = np.array(data[x_cols])
    Y = np.array(data.updated_final_activity)
    Y = pd.get_dummies(Y).values
    #print(X[-2:,])
    X = np.concatenate((X,X[-2:,]), axis =0)
    Y = np.concatenate((Y,Y[-2:,]), axis =0)
    #print(X[0:135,])
    i=0
    while i < len(data):
        #print(i)
        x = X[i:i+step,]
        y = Y[i:i+step,]
        #print(x.shape)
        i+= step
        agged.append(x)
        agged_y.append(y)
    agged = np.array(agged)
    agged_y = np.array(agged_y)
    return agged, agged_y

In [6]:
aggregate(thigh)[0].shape, aggregate(hip)[0].shape, aggregate(chest)[0].shape

((1511,), (1906,), (2015,))

In [7]:
# def reshape(agged,agged_y):
#     X_data = []
#     Y_data = []
#     for i in range(5):
#         x = agged[i]
#         y = agged_y[i]
#         num_rows = 167687- len(x)
#         if num_rows == 0:
#             X_data.append(x)
#             Y_data.append(y)
#             continue

#         sample = x[np.random.choice(x.shape[0], num_rows, replace=True)]
#         sd = np.std(sample, axis = 0)
#         mean = np.mean(sample,axis = 0)
#         sample = np.array([mean - x + np.random.normal()*sd/2 for x in sample])
#         #print(np.mean(sample,axis = 0))
#         sample_y = y[np.random.choice(y.shape[0], num_rows, replace=True)]
#         #print(sample.shape, x.shape)
#         #print(num_rows,len(x[np.random.choice(x.shape[0], num_rows, replace=True)]))
#         true_x = np.concatenate((x,sample),axis = 0)
#         true_y = np.concatenate((y,sample_y), axis = 0)
#         X_data.append(true_x)
#         Y_data.append(true_y)
#     X_data = np.array(X_data)
#     Y_data = np.array(Y_data)
#     return X_data, Y_data

This block runs the RNN on wrist with 2 LSTM layers and the data split into 45 second sequences and is run over 20 epochs.

In [35]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
x,y = aggregate(wrist,45)
# x_train = x[:int(len(x)*0.7),]
# x_test = x[int(len(x)*0.7):,]
# y_train = y[:int(len(x)*0.7),]
# y_test = y[int(len(x)*0.7):,]
X_train, X_test, y_train, y_test = train_test_split(x,y)
model = Sequential()
model.add(LSTM(128,input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
#model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences = True))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


model.fit(X_train,y_train, epochs = 20, batch_size = 50)
model.evaluate(X_test,y_test)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 45, 128)           108544    
_________________________________________________________________
lstm_19 (LSTM)               (None, 45, 64)            49408     
_________________________________________________________________
dropout_9 (Dropout)          (None, 45, 64)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 45, 32)            12416     
_________________________________________________________________
dense_55 (Dense)             (None, 45, 256)           8448      
_________________________________________________________________
dense_56 (Dense)             (None, 45, 128)           32896     
_________________________________________________________________
dropout_10 (Dropout)         (None, 45, 128)         

[0.5094038006625597, 0.7994139790534973]

This code block runs the RNN on hip with 2 LSTM layers with a timestep of 45 seconds and  20 epochs

In [36]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
x,y = aggregate(hip,45)
# x_train = x[:int(len(x)*0.7),]
# x_test = x[int(len(x)*0.7):,]
# y_train = y[:int(len(x)*0.7),]
# y_test = y[int(len(x)*0.7):,]
X_train, X_test, y_train, y_test = train_test_split(x,y)
model = Sequential()
model.add(LSTM(128,input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
#model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences = True))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


model.fit(X_train,y_train, epochs = 20, batch_size = 50)
model.evaluate(X_test,y_test)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 45, 128)           108544    
_________________________________________________________________
lstm_22 (LSTM)               (None, 45, 64)            49408     
_________________________________________________________________
dropout_11 (Dropout)         (None, 45, 64)            0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 45, 32)            12416     
_________________________________________________________________
dense_60 (Dense)             (None, 45, 256)           8448      
_________________________________________________________________
dense_61 (Dense)             (None, 45, 128)           32896     
_________________________________________________________________
dropout_12 (Dropout)         (None, 45, 128)         

[0.24158988169979947, 0.906258761882782]

759/759 [==============================] - 2s 3ms/step


[0.4090566622566957, 0.8281266689300537]

sit/lie represented half

In [14]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
x,y = aggregate(hip,100)
# x_train = x[:int(len(x)*0.7),]
# x_test = x[int(len(x)*0.7):,]
# y_train = y[:int(len(x)*0.7),]
# y_test = y[int(len(x)*0.7):,]
X_train, X_test, y_train, y_test = train_test_split(x,y)
model = Sequential()
model.add(LSTM(128,input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
#model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences = True))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


model.fit(X_train,y_train, epochs = 20, batch_size = 50)
model.evaluate(X_test,y_test)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 100, 128)          108544    
_________________________________________________________________
lstm_11 (LSTM)               (None, 100, 64)           49408     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 100, 32)           12416     
_________________________________________________________________
dense_16 (Dense)             (None, 100, 256)          8448      
_________________________________________________________________
dense_17 (Dense)             (None, 100, 128)          32896     
_________________________________________________________________
dropout_8 (Dropout)          (None, 100, 128)         

[0.11525638463982144, 0.960900604724884]

Wrist with sit/lie scaled back by 1/2

In [13]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
x,y = aggregate(wrist,100)
# x_train = x[:int(len(x)*0.7),]
# x_test = x[int(len(x)*0.7):,]
# y_train = y[:int(len(x)*0.7),]
# y_test = y[int(len(x)*0.7):,]
X_train, X_test, y_train, y_test = train_test_split(x,y)
model = Sequential()
model.add(LSTM(128,input_shape = (X_train.shape[1],X_train.shape[2]), return_sequences = True))
#model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences = True))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


model.fit(X_train,y_train, epochs = 20, batch_size = 50)
model.evaluate(X_test,y_test)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100, 128)          108544    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100, 64)           49408     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 100, 32)           12416     
_________________________________________________________________
dense_11 (Dense)             (None, 100, 256)          8448      
_________________________________________________________________
dense_12 (Dense)             (None, 100, 128)          32896     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 128)         

[0.1801559747376057, 0.9443633556365967]